In [3]:
import datetime as dt
import os
import warnings
from collections import UserDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 两种绘图接口
import plotly_express as px  
import plotly.graph_objects as go

%matplotlib inline

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(precision=2)

In [5]:
ts_data = pd.read_csv("../data/energy.csv")
ts_data = pd.DataFrame(ts_data[["load", "temp"]].values, index=pd.to_datetime(ts_data["Unnamed: 0"].values), columns=["load", "temp"])
ts_data = ts_data.asfreq(freq='H') #转换为小时频率的数据
ts_data.head()

,load,temp
2012-01-01 00:00:00,"2,698.00",32.00
2012-01-01 01:00:00,"2,558.00",32.67
2012-01-01 02:00:00,"2,444.00",30.00
2012-01-01 03:00:00,"2,402.00",31.00
2012-01-01 04:00:00,"2,403.00",32.00


In [8]:
#不包含时间维度的数据集划分
def split_dataset(df, test_size, valid_size, x_feature_list, y_feature_list):
    # 假设X是特征，y是标签
    X = df.copy()[x_feature_list]
    y = df.copy()[x_feature_list]
    # 划分为训练集和临时集合
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=42)
    # 进一步将临时集合划分为测试集和验证集
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=valid_size, random_state=42)
    
    #归一化
    X_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    
    #训练集
    X_train[x_feature_list] = X_scaler.fit_transform(X_train)
    X_train = X_train.values.astype('float32')
    #验证集
    X_val[x_feature_list] = X_scaler.fit_transform(X_val)
    X_val = X_val.values.astype('float32')
    #测试集
    X_test[x_feature_list] = X_scaler.fit_transform(X_test)
    X_test = X_test.values.astype('float32')
    
    #标签
    y_train[y_feature_list] = y_scaler.fit_transform(y_train)
    y_train = y_train.values.astype('float32')
    y_val[y_feature_list] = y_scaler.fit_transform(y_val)
    y_val = y_val.values.astype('float32')
    y_test[y_feature_list] = y_scaler.fit_transform(y_test)
    y_test = y_test.values.astype('float32')
    return y_scaler, X_train, X_val, X_test, y_train, y_val, y_test

In [9]:
y_scaler, X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(ts_data, 0.2, 0.3, ['load'], ['load'])

In [12]:
print("X_train shape: {0} y_train shape: {1}".format(X_train.shape, y_train.shape))
print("X_val shape: {0} y_val shape: {1}".format(X_val.shape, y_val.shape))
print("X_test shape: {0} y_test shape: {1}".format(X_test.shape, y_test.shape))

X_train shape: (21043, 1) y_train shape: (21043, 1)
X_val shape: (1579, 1) y_val shape: (1579, 1)
X_test shape: (3682, 1) y_test shape: (3682, 1)
